# Database Cleanser

This notebook contains tools to cleanup the database and remove testing models and indicators.

In [ ]:
from elasticsearch import Elasticsearch
import requests
import json
import os

In [ ]:
es = Elasticsearch('ES_HERE')
causemos_user = 'worldmodelers'
causemos_pwd = "PWD_HERE"

## Delete by ID

Delete model and model artifacts by ID

In [ ]:
def search_by_model(model_id):
    q = {"query": {"term": {"model_id.keyword": {"value": model_id, "boost": 1.0}}}}
    return q

def del_by_id(model_id):
    try:
        print(es.delete(index='models', id=model_id))
    except:
        pass

    # delete directives
    results = es.search(index="directives", body=search_by_model(model_id))
    print(results)
    for directive in results.get('hits',{}).get('hits',[]):
        if 'id' in directive:
            k_ = 'id'
        else:
            k_ = '_id'
        print(es.delete(index='directives', id=directive[k_]))

    # delete configs            
    results = es.search(index="configs", body=search_by_model(model_id))
    print(results)        
    for config in results.get('hits',{}).get('hits',[]):
        if 'id' in config:
            k_ = 'id'
        else:
            k_ = '_id'        
        print(es.delete(index='configs', id=config[k_]))

    # delete outputfiles            
    results = es.search(index="outputfiles", body=search_by_model(model_id))
    print(results)        
    for of in results.get('hits',{}).get('hits',[]):
        if 'id' in of:
            k_ = 'id'
        else:
            k_ = '_id'                
        print(es.delete(index='outputfiles', id=of[k_]))    
        
    # delete runs
    results = es.search(index="runs", body=search_by_model(model_id))
    print(results)
    for run in results.get('hits',{}).get('hits',[]):
        if 'id' in run:
            k_ = 'id'
        else:
            k_ = '_id'                
        print(es.delete(index='outputfiles', id=run[k_]))            

In [ ]:
model_id = input()

In [ ]:
model_id = model_id.strip()

In [ ]:
print(model_id)
del_by_id(model_id)

## Notify Uncharted of New Model

In [ ]:
def notify_causemos(model):
    headers = {"accept": "application/json", "Content-Type": "application/json"}
    
    url = 'https://causemos.uncharted.software/api/maas/datacubes'

    response = requests.post(
        url,
        headers={"Content-Type": "application/json"},
        json=model,
        auth=(causemos_user, causemos_pwd),
    )

def submit_run(model):
    headers = {"accept": "application/json", "Content-Type": "application/json"}
    url = 'https://causemos.uncharted.software/api/maas/model-runs'

    params = []
    for param in model.get("parameters",[]):
        param_obj = {}
        param_obj['name'] = param['name']
        param_obj['value'] = param['default']
        params.append(param_obj)

    payload = {"model_id": model["id"],
               "model_name": model["name"],
               "is_default_run": True,
               "parameters": params}

    response = requests.post(
        url,
        headers={"Content-Type": "application/json"},
        json=payload,
        auth=(causemos_user, causemos_pwd),
    )

In [ ]:
model_id = input()

In [ ]:
model_id = model_id.strip()

In [ ]:
model = es.get(index='models', id=model_id)['_source']

In [ ]:
notify_causemos(model)

In [ ]:
submit_run(model)